In [ ]:
from google.colab import drive
drive.mount("/content/drive")
%cd /content/drive/MyDrive/HEDSpython

# **Pandas**

Pandas is an open source Python library for data analysis. Most of the times it is pre-installed in your system, but, as for any other library, you need to import it.

Let's start with importing pandas.

In [ ]:
import pandas as pd

Let's import a dataset. To do so we use the `read_csv()` function.

In [ ]:
csvFilePath = 'files/internallyDisplaced.csv'
internallyDisplaced = pd.read_csv(csvFilePath)

You can also read excel files with the `read_excel()` function. The rest is the same you just need to define the right file path.

## **Inspecting a dataframe**

The methods `.head()` and `.tail()` show respectively the first and last 5 rows of a dataframe.

In [ ]:
internallyDisplaced.head()

In [ ]:
internallyDisplaced.tail()

You can also give the number of rows that you want to show as an argument to the method.

In [ ]:
internallyDisplaced.head(10)

In [ ]:
internallyDisplaced.tail(10)

The attribute `.shape` returns the dimensions (rows, columns) of the dataframe.

In [ ]:
internallyDisplaced.shape

You can use the `.sample()` method to show a random number of rows.

In [ ]:
internallyDisplaced.sample(10)

If you re-run the line of code above you will see that the selection of rows will change.

## **Selecting columns and rows in Pandas**

Another way to check which columns are in the dataframe is

In [ ]:
internallyDisplaced.columns

You can select a single column in the following way.

In [ ]:
internallyDisplaced['Name']

Or in the following way

In [ ]:
internallyDisplaced.Name

To select multiple columns you need to parse the column names as a list within the first [].

An example

In [ ]:
internallyDisplaced[['Hazard Type', 'Name']]

You can compare column items against a certain value. Doing so will return `True` or `False` for each item.

In [ ]:
internallyDisplaced['Hazard Type'] == 'Flood'

You can use such comparison to select specific rows in a dataframe (i.e., those that meet the stated condition).

In [ ]:
internallyDisplaced[internallyDisplaced['Hazard Type'] == 'Flood']

You can also combine different conditions.

In [ ]:
internallyDisplaced[(internallyDisplaced['Hazard Type'] == 'Flood')| (internallyDisplaced['Hazard Type'] == 'Storm')]

Note remember the () inside or the code will throw an error.

You can use the following operators to combine conditions:


*   & to AND
*   | to OR
*   ~ to NEGATE

In [ ]:
internallyDisplaced[(internallyDisplaced['Hazard Type'] == 'Flood') & (internallyDisplaced['Name'] == 'Zimbabwe')]

In [ ]:
internallyDisplaced[(internallyDisplaced['Hazard Type'] == 'Flood') & ~ (internallyDisplaced['Name'] == 'Zimbabwe')]

Try to select the event name, year and type of hazard that happened in the UK after 2010.

In [ ]:
# insert your code below.
# suggestion: first select hazard that happened after 2010 in the UK and then select columns.

You can also index over a subset of a dataframe. For instance, here is an alternative way to select the first or last 10 rows of a subset of a dataframe.

In [ ]:
internallyDisplaced['New Displacements'][:10] # first 10

In [ ]:
internallyDisplaced['New Displacements'][-10:] # last 10

You can store subset of dataframes on separate variables and use all methods presented above on them.

Now let's introduce some new methods that are useful for numeric columns to exemplify.

`.min()` gives you the minimum value.

In [ ]:
internallyDisplaced['New Displacements'].min()

`.max()` gives you the maximum value.

In [ ]:
internallyDisplaced['New Displacements'].max()

.`mean()` gives you the mean value.

In [ ]:
internallyDisplaced['New Displacements'].mean()

`.describe()` gives you the summary statistics for numeric columns in dataframe.

In [ ]:
internallyDisplaced.describe()

You can use `.loc` and `.iloc` to select a specific item in a dataframe. The only difference between the two is that `.loc` takes rows and columns names as arguments while `.iloc` takes their index.

Note, in this case, the row names are indexes.

In [ ]:
internallyDisplaced.loc[1, 'Name']

In [ ]:
internallyDisplaced.iloc[1, 1]

You can create a new column by simply assigning it to the column name.

In [ ]:
internallyDisplaced['Before_2010'] =  internallyDisplaced['Year'] <= 2010
internallyDisplaced[['Event Name','Before_2010']]

Now create a column to identify events that happened between 2010 and 2012.

In [ ]:
# insert your code here

The method `.groupby()` is used to split a dataframe into groups.

Let's say you want to know how many people were displaced by a specific hazard before and after 2010.

In [ ]:
internallyDisplaced.groupby(['Hazard Type', 'Before_2010'])['New Displacements'].sum()

You can also use `.groupby()` to know how many instances of a class are in the dataframe.  

In [ ]:
internallyDisplaced.groupby(['Hazard Type'])['Hazard Type'].count()

When was the latest hazard recorded by country?

In [ ]:
# insert code here

## **Dealing with missing data**

The method `.isna()` tells you which items are `NaN` in your dataframem combining it with `.sum()` allows us to know how many missing values there are per column.

In [ ]:
internallyDisplaced.isna().sum()

We can deal with missing data in our dataframe by replacing them with some fix value (i.e., the sample mean).

In [ ]:
internallyDisplaced.fillna(internallyDisplaced.mean())

Note that you need to reassign the dataframe to `internallyDisplaced` running `internallyDisplaced = internallyDisplaced.fillna(internallyDisplaced.mean())` or your changes will not be reflected in the dataframe.

In [ ]:
internallyDisplaced

You can also choose to drop incomplete rows using `.dropna()`.

In [ ]:
internallyDisplaced.dropna()

You can also:

1.   Only drop rows where all values are `NaN` values using `.dropna(how='all')`
2.   Put a threshold to how many non null values need to be in a row in order to keep it, using `.dropna(thresh=10)`
3.   Choose for which column `NaN` values should be counted while dropping using `.dropna(subset=['New Displacements'])`.

An example:


In [ ]:
internallyDisplaced.dropna(subset=['New Displacements'])

##**Data Manipulation and Visualization**

Now let's go over some example of data manipulation and visualization.

First, as usual, we import relevant libraries.

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import re

Take a look at the `emdat_EQ_FL_LA_1900-2022_world.xlsx` file in the files folder. You can see that this presents a header in the first 6 lines so we have to make sure that the first 6 lines are skipped when reading the excel. We can create a function that does this for us.

In [ ]:
def read_EMDAT(emdatXlsFilePath):
    rowsToSkip = range(6)
    rawData = pd.read_excel(emdatXlsFilePath, skiprows=rowsToSkip)
    return rawData

We can now run the function using our file path as an argument.

In [ ]:
emdatFilePath = 'files/emdat_EQ_FL_LA_1900-2022_world.xlsx'
impactData = read_EMDAT(emdatFilePath)

We can now check what unique disaster types are reported in the dataframe.

In [ ]:
hazTypesGeneral = pd.unique(impactData['Disaster Type'])
hazTypesGeneral

Yes, you can use `.unique()` both as a method and as a function. The result is exactly the same.

Let's create a pie chart by event type in the dataframe and of events that happened after 1970.

We first identify how many disasters of each type there are and sore them in a list.

In [ ]:
numHazTypeGeneral = []
splitYear = 1970
numHazTypeGeneralAfterYear = []
for hazType in hazTypesGeneral:
  num_hazrds = sum(impactData['Disaster Type'] == hazType)
  numHazTypeGeneral.append(num_hazrds)
  num_hazards_post_1970 = sum(impactData['Disaster Type'].eq(hazType) & \
                                          impactData['Start Year'].ge(splitYear))
  numHazTypeGeneralAfterYear.append(num_hazards_post_1970)
print(numHazTypeGeneral)
print(numHazTypeGeneralAfterYear)

We then plot the values using the pyplot library.

In [ ]:
plt.figure(dpi=200)
plt.pie(numHazTypeGeneral, labels=hazTypesGeneral, autopct='%1.1f%%')
plt.show()

We can do the same for events after 1970.

In [ ]:
plt.figure(dpi=200)
plt.pie(numHazTypeGeneralAfterYear, labels=numHazTypeGeneralAfterYear, autopct='%1.1f%%')
plt.show()

We now try to create two plots side by side, one with the overall hazard distribution overtime and the other with a bar chart reporting the the split in time of floods events before and after 1970.

In [ ]:
# make figure and assign axis objects
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(9, 5), dpi=200)
fig.subplots_adjust(wspace=0)

# pie chart parameters
explode = [0 for x in range(len(hazTypesGeneral))]

ax1.pie(numHazTypeGeneral, autopct='%1.1f%%', labels=hazTypesGeneral, explode=explode)

# bar chart parameters
hazToSplit = 2
specific_ratios = [numHazTypeGeneral[hazToSplit]-numHazTypeGeneralAfterYear[hazToSplit],
                   numHazTypeGeneral[hazToSplit]]
specific_labels = ['Before '+str(splitYear), str(splitYear)+' onwards']

width = 0.2
bc = ax2.bar(0, specific_ratios[0], width, bottom=None,
             label=specific_labels[0], color='C'+str(hazToSplit), alpha= 0.3)
ax2.bar_label(bc, labels=[f"{specific_ratios[0]}"], label_type='center')
bc = ax2.bar(0, specific_ratios[1], width, bottom=specific_ratios[0],
             label=specific_labels[1], color='C2'+str(hazToSplit), alpha= 0.6)
ax2.bar_label(bc, labels=[f"{specific_ratios[1]}"], label_type='center')

ax2.set_title(hazTypesGeneral[hazToSplit])
ax2.legend()
ax2.axis('off')
ax2.set_xlim(- 2.5 * width, 2.5 * width)

plt.show()

We can also plot events overtime.

In [ ]:
minYear = min(impactData['Start Year'])
maxYear = max(impactData['Start Year'])

numEventsOneYear = []
for yr in range(minYear, maxYear+1):
    numEventsOneYear.append(sum(impactData['Start Year'] == yr))

plt.figure(dpi=200)
plt.bar(range(minYear, maxYear+1), numEventsOneYear)
plt.xlabel('Year')
plt.ylabel('Number of events [-]')
plt.show()

We can also plot the frequency by event type.

In [ ]:
# disaggregated events
numEventsOneYearDisagg = dict()
for hazType in hazTypesGeneral:
    numEventsOneYearDisagg[hazType] = []
    for yr in range(minYear, maxYear+1):
        numEventsOneYearDisagg[hazType].append(
            sum(impactData['Start Year'].eq(yr) &
                impactData['Disaster Type'].eq(hazType) ) )

plt.figure(dpi=200)
prevSeries = None
for hazType in hazTypesGeneral:
    plt.bar(range(minYear, maxYear+1), numEventsOneYearDisagg[hazType],
            label=hazType, bottom=prevSeries)
    prevSeries = numEventsOneYearDisagg[hazType]

plt.legend()
plt.xlabel('Year')
plt.ylabel('Number of events [-]')
plt.show()

We can also map the events using the given latitude and longitude values.

First, we only retain values where longitude and latutitude are not NaN.

In [ ]:
impactData = impactData.dropna(subset=['Longitude', 'Latitude'])

Let's check that we did this successfully.

In [ ]:
impactData[['Longitude','Latitude']]

Now let's convert latitude and longitude values into floats.

In [ ]:
for index, row in impactData.iterrows():

    try:
        # proper number or nan
        impactData['Longitude'][index]= float(row['Longitude'])
    except:
        # string
        impactData['Longitude'][index] = float(re.findall(r"[-+]?(?:\d*\.\d+|\d+)", row['Longitude'])[0])

    try:
        # proper number or nan
        impactData['Latitude'][index] = float(row['Latitude'])
    except:
        # string
        impactData['Latitude'][index] = float(re.findall(r"[-+]?(?:\d*\.\d+|\d+)", row['Latitude'])[0])

#impactData[['Latitude', 'Longitude']]

In [ ]:
hazTypesGeneral = pd.unique(impactData['Disaster Type']).tolist()
hazTypesGeneral.sort(reverse = True)
hazTypesGeneral

We then map them using the resulting coordinates.

In [ ]:
plt.figure(dpi=200)

for ind,hazType in enumerate(hazTypesGeneral):
    impactSliced = impactData[impactData['Disaster Type'] == hazType]
    plt.scatter(impactSliced['Longitude'], impactSliced['Latitude'],
                c='C'+str(ind), alpha=0.5, edgecolors='none',
                label=hazType)

plt.legend()
plt.xlabel('Longitude [°]')
plt.ylabel('Latitude [°]')
plt.show()

We can also loop over the hazard types to create separate equivalent plots by hazard.

In the example below we generate three different maps for each hazard scaling the dimensions of each event location by total deaths.  

In [ ]:
scaleFactor = [0.5, 0.01]
for ind,hazType in enumerate(hazTypesGeneral):
  plt.figure(dpi=200)
  impactSliced = impactData[impactData['Disaster Type'] == hazType]
  plt.scatter(impactSliced['Longitude'], impactSliced['Latitude'],
              c='C'+str(ind), alpha=0.5, edgecolors='none',
              label=hazType, s=impactSliced['Total Deaths']*scaleFactor[ind])

  plt.title(hazType)
  plt.xlabel('Longitude [°]')
  plt.ylabel('Latitude [°]')
  plt.show()